In [6]:
import sys
stdout = sys.stdout
import numpy as np
from numpy.random import shuffle
sys.stdout = stdout
from keras.utils.np_utils import to_categorical
import time
print("started")

started


In [29]:
class Network(object):
    def __init__(
        self,
        nodes,
        step_size=0
    ):
        """
        nodes represents the number of nodes per layer. 
        eg: [2,3,5] is 2 feature input, 
        3 neurons in the first layer,
        5 neurons in the final layer.
        5 also represents the number of classes
        
        nb_layers includes both the output and input layers
        outputs is the output matrix of each layer. An output matrix is of shape (p, n),
            where p is the number of examples given to the feedforward, and n is the nb of nodes in the layer
        derivates is the derivates of each layer. Each row is a different layer
        errod_ds is the derivate of the error function
        
        Default cost function set to cross entropy, therefore, weights[-1] == nb of classes
        """
        self.step_size = step_size
        
        self.nb_layers = len(nodes)
        self.nodes = nodes
        self.weights = [ np.ones((n+1, m), dtype=float) for n,m in zip(nodes[:-1], nodes[1:])]
        self.outputs = [ 0 for n in nodes ]
        self.derivatives = [ 0 for n in nodes[1:]]
        self.error_ds = np.zeros(nodes[-1])
        self.gradients = [ 0 for n in nodes[1:]]
        
    def fforw(self, inputs, labels):
        shape = np.shape(inputs)
        self.outputs[0] = np.array(inputs)
        cur_input = np.ones((shape[0], shape[1] + 1))
        cur_input[:, :-1] = inputs
        for l,w in enumerate(self.weights):
            out = self.relu(np.dot(cur_input, w))
#             print(l, out)
            self.outputs[l + 1] = out
            self.derivatives[l] = self.relu(out, True)
#             print("sum derivative #" + str(l) + " out: " + str(np.sum(out)) +" :SD "+ str(np.sum(self.derivatives[l])))
            
            shape = np.shape(out)
            cur_input = np.ones((shape[0], shape[1] + 1))
            cur_input[:, :-1] = out
        
        #Calculating the derivative of the error function for backprop
        self.error_ds = self.softmaxLoss(cur_input[:, :-1], labels)
        
        #appending the input as output[-1] for future use
#         self.outputs.append([])
        
        return   

    def backprop(self):
        """
        Each gradient is of shape p x m,
            where p is the number of examples, m is the number of output nodes from the layer
        """
        self.gradients[-1] = self.derivatives[-1] * self.error_ds
        
        for i in range(1, len(self.derivatives)):
            index = len(self.derivatives) - 1 - i
            example_gradients = np.zeros(np.shape(self.derivatives[index]))
            for j,example in enumerate(self.derivatives[index]):
                example_gradients[j] = np.dot(np.diag(self.derivatives[index][j]), self.weights[index + 1][:-1]).dot(self.gradients[index+1][j])        
            self.gradients[index] = example_gradients
    
    def update_weights(self, inputs, labels):
        """
        inputs is the given input for the network.
        Shape of inputs should be (n x m)
            Where n is the number of examples,
            m is the number of features
        labels are the correct labels for each example of shape (n,)
        """
        self.fforw(inputs, labels)
        self.backprop()
        for i,w in enumerate(self.weights):
#             print(i)
            shape = np.shape(self.outputs[i])
            hat_o = np.ones((shape[0], shape[1] + 1))
            hat_o [:, :-1] = self.outputs[i]
            for e,g in enumerate(self.gradients[i]):
                single_grad = self.gradients[i][e]
                single_grad.shape = (len(single_grad), 1)
                single_hat_o = hat_o[e]
                single_hat_o.shape = (1, len(single_hat_o))
#                 print(np.sum(single_grad))
                single_update = -self.step_size*(np.dot(single_grad, single_hat_o)).T
                self.weights[i] += single_update
        
    def relu(self, x, derivative=False):
        if derivative:
            result = x.copy()
            result[result<=0] = 0
            result[result>0] = 1
            return result
        else:
            return x * (x > 0)
        
    def sigmoid(self, x, derivative=False):
        if derivative:
            return x*(1-x)
        else:
            return 1/(1+np.exp(-x))

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
    
    def softmaxLoss(self, X, y):
        if (len(y.shape) == 0):
            m = 1
        else:
            m = y.shape[0]
        p = self.softmax(X)
#         log_likelihood = -np.log(p[range(m), y])
#         loss = np.sum(log_likelihood) / m

        dx = p.copy()
        dx[range(m), y] -= 1
        dx /= m
        return dx
    
    def fit(self, inputs, labels, epochs, batch_size=32):
        t_start = time.clock()
        print("starting fit, time is: ", t_start)
        t_last = time.clock()
        for i in range(epochs):
            print("epoch: ", i + 1)
            # randomize data
#             print(np.shape(inputs), np.shape(labels))
            state = np.random.get_state()
            np.random.shuffle(inputs)
            np.random.set_state(state)
            np.random.shuffle(labels)
#             print(np.shape(inputs), np.shape(labels))
            batches = np.floor_divide(len(inputs), batch_size)
            for b in range(batches):
                print("batch: ", b)
                start = b * batch_size
                end = (b+1) * batch_size
                self.update_weights(inputs[start:end], labels[start:end])
            self.update_weights(inputs[batches*batch_size:], labels[batches*batch_size:])
            t_epoch = time.clock()
#             print("time elapsed is: ", t_epoch - t_last)
            
    def predict(self, inp, labels):
        self.fforw(inp, labels)
#         return self.outputs[len(self.nodes) - 2]
        print(self.outputs[-1])
        return [np.argmax(x) for x in self.outputs[len(self.nodes) - 1]] #output of final layer

# General functions

def safe_divide(num, denom):
    if len(num) != len(denom):
        return []
    return [num[i] / denom[i] if num[i] > 0.0 and denom[i] > 0.0 else 0.0 for i in range(len(num))]

def score(preds, targets, nclasses):
    # calculate fscore for each class, then take macro average (unweighted)
    if (len(preds) != len(targets)):
        return
    true_pos = [0.0] * nclasses
    pos = [0.0] * nclasses
    rel_pos = [0.0] * nclasses
    for i in range(len(preds)):
        if preds[i] == targets[i]:
            true_pos[targets[i]] += 1.0
        pos[preds[i]] += 1.0
        rel_pos[targets[i]] += 1.0
    
    true_pos = np.array(true_pos)
    pos = np.array(pos)
    rel_pos = np.array(rel_pos)
#     print(true_pos, pos, rel_pos)
    p = safe_divide(true_pos, pos)
    r = safe_divide(true_pos, rel_pos)
#     print(p, r)
    
    f_scores = [2.0 * p[i] * r[i] / (p[i] + r[i]) if p[i] + r[i] > 0.0 else 0.0 for i in range(len(p))]
    
    return np.mean(f_scores)

In [8]:
# Example of 1 hidden layer, 1 output layer NN. 
# Input is 2 features, nb of classes is 5 in this case
n = Network([3,3,2])

start_time = time.clock()

# Updates the weights given a mini-batch of 2 examples in this case 
    #Eg: [1,2] are the features for the first example, [3,4] are the features for the second example
    #[0,4] represents the labels of the given examples, where 0 means [1,2] represents class 1, 
    # and 4 means [3,4] is class 5.
# n.update_weights([[1,2], [3,4]], np.array([0,4]))

In [131]:
print("Test Example")
data = [[2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2],[100, 100, 100],[100, 100, 100], [100, 100, 100], [100, 100, 100]]
targets = [0,0,0,0,1,1,1,1]
print("We train the network on the following: ")
print("data: ", data, "\ntargets: ", targets)
n.fit(data, np.array(targets), 1000)
print("Results:")
test = [[2, 2, 2], [2,2,2], [2,2,2], [100,100,100], [100,100,100], [100,100,100]]
test_targ = [0, 0, 0, 1, 1, 1]
preds = n.predict(test, np.array(test_targ))
print("tests: ", test, "predictions: ", preds, "score: ", score(preds, test_targ, 2))


Test Example
We train the network on the following: 
data:  [[2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [100, 100, 100], [100, 100, 100], [100, 100, 100], [100, 100, 100]] 
targets:  [0, 0, 0, 0, 1, 1, 1, 1]
starting fit, time is:  1201.393698
epoch:  1
0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
epoch:  2
0
1.548465622444968e-10
1.548465622444968e-10
0.0
0.0
0.0
1.548465622444968e-10
1.548465622444968e-10
0.0
1
7.356541107598862e-08
7.356541107598862e-08
-7.342359491517743e-08
-7.342359491517743e-08
-7.342359491517743e-08
7.356541107598862e-08
7.356541107598862e-08
-7.342359491517743e-08
epoch:  3
0
3.0959994564008467e-10
0.0
0.0
0.0
0.0
3.0959994564008467e-10
3.0959994564008467e-10
3.0959994564008467e-10
1
1.4708474411128866e-07
-1.468013759141091e-07
-1.468013759141091e-07
-1.468013759141091e-07
-1.468013759141091e-07
1.4708474411128866e-07
1.4708474411128866e-07
1.4708474411128866e-07
epoch:  4
0
0.0
4.6426021067952733e-10
4.6426021067952733e-10
0.0
4.64260

-5.801316058225755e-06
5.811979882168342e-06
5.811979882168342e-06
-5.801316058225755e-06
-5.801316058225755e-06
epoch:  83
0
1.239325799678549e-08
0.0
1.239325799678549e-08
0.0
1.239325799678549e-08
0.0
1.239325799678549e-08
0.0
1
5.881906507917024e-06
-5.8711210398705756e-06
5.881906507917024e-06
-5.8711210398705756e-06
5.881906507917024e-06
-5.8711210398705756e-06
5.881906507917024e-06
-5.8711210398705756e-06
epoch:  84
0
1.2540661210777473e-08
1.2540661210777473e-08
0.0
1.2540661210777473e-08
0.0
0.0
1.2540661210777473e-08
0.0
1
5.951789372652229e-06
5.951789372652229e-06
-5.940882494624104e-06
5.951789372652229e-06
-5.940882494624104e-06
-5.940882494624104e-06
5.951789372652229e-06
-5.940882494624104e-06
epoch:  85
0
1.2687976071829353e-08
0.0
1.2687976071829353e-08
0.0
1.2687976071829353e-08
1.2687976071829353e-08
0.0
0.0
1
6.021628504196538e-06
-6.0106004499810555e-06
6.021628504196538e-06
-6.0106004499810555e-06
6.021628504196538e-06
6.021628504196538e-06
-6.0106004499810555e-0

2.419189800386285e-08
2.419189800386285e-08
0.0
1
1.1469493646279298e-05
-1.1449498817911637e-05
1.1469493646279298e-05
-1.1449498817911637e-05
-1.1449498817911637e-05
1.1469493646279298e-05
1.1469493646279298e-05
-1.1449498817911637e-05
epoch:  166
0
2.433224276764329e-08
0.0
0.0
2.433224276764329e-08
0.0
0.0
2.433224276764329e-08
2.433224276764329e-08
1
1.1535881102233943e-05
-1.151578291454171e-05
-1.151578291454171e-05
1.1535881102233943e-05
-1.151578291454171e-05
-1.151578291454171e-05
1.1535881102233943e-05
1.1535881102233943e-05
epoch:  167
0
2.447250374004573e-08
0.0
2.447250374004573e-08
0.0
0.0
2.447250374004573e-08
0.0
2.447250374004573e-08
1
1.1602227044331349e-05
-1.158202570398061e-05
1.1602227044331349e-05
-1.158202570398061e-05
-1.158202570398061e-05
1.1602227044331349e-05
-1.158202570398061e-05
1.1602227044331349e-05
epoch:  168
0
0.0
2.4612680975078694e-08
0.0
0.0
2.4612680975078694e-08
0.0
2.4612680975078694e-08
2.4612680975078694e-08
1
-1.1648227212196494e-05
1.1668

3.556027235764464e-08
3.556027235764464e-08
3.556027235764464e-08
3.556027235764464e-08
0.0
1
-1.681286940759816e-05
-1.681286940759816e-05
-1.681286940759816e-05
1.684076775611558e-05
1.684076775611558e-05
1.684076775611558e-05
1.684076775611558e-05
-1.681286940759816e-05
epoch:  249
0
3.569384310289813e-08
0.0
0.0
3.569384310289813e-08
0.0
3.569384310289813e-08
0.0
3.569384310289813e-08
1
1.690379756078518e-05
-1.687581198021957e-05
-1.687581198021957e-05
1.690379756078518e-05
-1.687581198021957e-05
1.690379756078518e-05
-1.687581198021957e-05
1.690379756078518e-05
epoch:  250
0
0.0
0.0
0.0
3.58273344219666e-08
3.58273344219666e-08
3.58273344219666e-08
0.0
3.58273344219666e-08
1
-1.6938715347746295e-05
-1.6938715347746295e-05
-1.6938715347746295e-05
1.6966787978276148e-05
1.6966787978276148e-05
1.6966787978276148e-05
-1.6938715347746295e-05
1.6966787978276148e-05
epoch:  251
0
0.0
0.0
3.596074636583212e-08
0.0
3.596074636583212e-08
3.596074636583212e-08
3.596074636583212e-08
0.0
1
-1

0.0
0.0
0.0
4.625376589303211e-08
4.625376589303211e-08
1
2.1880585367447386e-05
-2.1846114784649512e-05
2.1880585367447386e-05
-2.1846114784649512e-05
-2.1846114784649512e-05
-2.1846114784649512e-05
2.1880585367447386e-05
2.1880585367447386e-05
epoch:  331
0
4.638099042653928e-08
4.638099042653928e-08
0.0
0.0
0.0
4.638099042653928e-08
4.638099042653928e-08
0.0
1
2.1940466749118112e-05
2.1940466749118112e-05
-2.190592297540185e-05
-2.190592297540185e-05
-2.190592297540185e-05
2.1940466749118112e-05
2.1940466749118112e-05
-2.190592297540185e-05
epoch:  332
0
0.0
0.0
4.650813961298195e-08
0.0
4.650813961298195e-08
4.650813961298195e-08
0.0
4.650813961298195e-08
1
-2.1965693929498374e-05
-2.1965693929498374e-05
2.2000310733653102e-05
-2.1965693929498374e-05
2.2000310733653102e-05
2.2000310733653102e-05
-2.1965693929498374e-05
2.2000310733653102e-05
epoch:  333
0
0.0
4.66352135006969e-08
0.0
0.0
4.66352135006969e-08
4.66352135006969e-08
0.0
4.66352135006969e-08
1
-2.2025427670327056e-05
2.

...

3.9841824054820136e-05
-3.979052321593092e-05
-3.979052321593092e-05
3.9841824054820136e-05
3.9841824054820136e-05
-3.979052321593092e-05
epoch:  663
0
0.0
0.0
0.0
8.473404279208152e-08
8.473404279208152e-08
0.0
8.473404279208152e-08
8.473404279208152e-08
1
-3.9839166972382815e-05
-3.9839166972382815e-05
-3.9839166972382815e-05
3.989049954292254e-05
3.989049954292254e-05
-3.9839166972382815e-05
3.989049954292254e-05
3.989049954292254e-05
epoch:  664
0
0.0
0.0
8.483866524008639e-08
8.483866524008639e-08
0.0
0.0
8.483866524008639e-08
8.483866524008639e-08
1
-3.988778048482841e-05
-3.988778048482841e-05
3.99391446928648e-05
3.99391446928648e-05
-3.988778048482841e-05
-3.988778048482841e-05
3.99391446928648e-05
3.99391446928648e-05
epoch:  665
0
8.494322680124692e-08
0.0
8.494322680124692e-08
8.494322680124692e-08
0.0
0.0
8.494322680124692e-08
0.0
1
3.998775952374469e-05
-3.9936363772221945e-05
3.998775952374469e-05
3.998775952374469e-05
-3.9936363772221945e-05
-3.9936363772221945e-05
3.99

0.0
0.0
1
4.378037930451574e-05
4.378037930451574e-05
-4.3726751131222116e-05
4.378037930451574e-05
4.378037930451574e-05
-4.3726751131222116e-05
-4.3726751131222116e-05
-4.3726751131222116e-05
epoch:  746
0
0.0
0.0
0.0
9.32140300563841e-08
0.0
9.32140300563841e-08
9.32140300563841e-08
9.32140300563841e-08
1
-4.3772947929947075e-05
-4.3772947929947075e-05
-4.3772947929947075e-05
4.382660053418104e-05
-4.3772947929947075e-05
4.382660053418104e-05
4.382660053418104e-05
4.382660053418104e-05
epoch:  747
0
9.331372875170559e-08
9.331372875170559e-08
9.331372875170559e-08
0.0
9.331372875170559e-08
0.0
0.0
0.0
1
4.3872792968263456e-05
4.3872792968263456e-05
4.3872792968263456e-05
-4.381911601493904e-05
4.3872792968263456e-05
-4.381911601493904e-05
-4.381911601493904e-05
-4.381911601493904e-05
epoch:  748
0
0.0
0.0
9.341336969883684e-08
9.341336969883684e-08
9.341336969883684e-08
0.0
0.0
9.341336969883684e-08
1
-4.386525540418383e-05
-4.386525540418383e-05
4.391895662488607e-05
4.391895662488

0.0
1.0110561988586043e-07
0.0
1.0110561988586043e-07
0.0
1
-4.7421100886819913e-05
4.7476472573103985e-05
4.7476472573103985e-05
-4.7421100886819913e-05
4.7476472573103985e-05
-4.7421100886819913e-05
4.7476472573103985e-05
-4.7421100886819913e-05
epoch:  828
0
1.0120075816994126e-07
0.0
0.0
0.0
1.0120075816994126e-07
1.0120075816994126e-07
0.0
1.0120075816994126e-07
1
4.752039165331146e-05
-4.74650018018987e-05
-4.74650018018987e-05
-4.74650018018987e-05
4.752039165331146e-05
4.752039165331146e-05
-4.74650018018987e-05
4.752039165331146e-05
epoch:  829
0
0.0
1.0129584161002766e-07
1.0129584161002766e-07
0.0
0.0
0.0
1.0129584161002766e-07
1.0129584161002766e-07
1
-4.750887543875982e-05
4.756428338426933e-05
4.756428338426933e-05
-4.750887543875982e-05
-4.750887543875982e-05
-4.750887543875982e-05
4.756428338426933e-05
4.756428338426933e-05
epoch:  830
0
0.0
1.0139087024093398e-07
1.0139087024093398e-07
1.0139087024093398e-07
0.0
1.0139087024093398e-07
0.0
0.0
1
-4.755272181445735e-05
4

0.0
1.087278746761862e-07
1.087278746761862e-07
0.0
0.0
0.0
1.087278746761862e-07
1
5.098851281070153e-05
-5.093187683452268e-05
5.098851281070153e-05
5.098851281070153e-05
-5.093187683452268e-05
-5.093187683452268e-05
-5.093187683452268e-05
5.098851281070153e-05
epoch:  910
0
0.0
0.0
0.0
1.0881862971347136e-07
0.0
1.0881862971347136e-07
1.0881862971347136e-07
1.0881862971347136e-07
1
-5.0973596641218366e-05
-5.0973596641218366e-05
-5.0973596641218366e-05
5.10302453538441e-05
-5.0973596641218366e-05
5.10302453538441e-05
5.10302453538441e-05
5.10302453538441e-05
epoch:  911
0
0.0
0.0
0.0
1.0890933269575847e-07
1.0890933269575847e-07
1.0890933269575847e-07
0.0
1.0890933269575847e-07
1
-5.101529053333953e-05
-5.101529053333953e-05
-5.101529053333953e-05
5.107195192099621e-05
5.107195192099621e-05
5.107195192099621e-05
-5.101529053333953e-05
5.107195192099621e-05
epoch:  912
0
1.0899998365611424e-07
1.0899998365611424e-07
0.0
1.0899998365611424e-07
0.0
0.0
1.0899998365611424e-07
0.0
1
5.11

0.0
0.0
1.1599980851341015e-07
0.0
1.1599980851341015e-07
0.0
1
5.4325741977330304e-05
5.4325741977330304e-05
-5.4268256123864416e-05
-5.4268256123864416e-05
5.4325741977330304e-05
-5.4268256123864416e-05
5.4325741977330304e-05
-5.4268256123864416e-05
epoch:  992
0
0.0
1.16086403139368e-07
0.0
1.16086403139368e-07
0.0
1.16086403139368e-07
1.16086403139368e-07
0.0
1
-5.430790386121869e-05
5.436539776662065e-05
-5.430790386121869e-05
5.436539776662065e-05
-5.430790386121869e-05
5.436539776662065e-05
5.436539776662065e-05
-5.430790386121869e-05
epoch:  993
0
0.0
0.0
1.1617294835588043e-07
1.1617294835588043e-07
0.0
1.1617294835588043e-07
0.0
1.1617294835588043e-07
1
-5.434752697961831e-05
-5.434752697961831e-05
5.4405028884072426e-05
5.4405028884072426e-05
-5.434752697961831e-05
5.4405028884072426e-05
-5.434752697961831e-05
5.4405028884072426e-05
epoch:  994
0
0.0
0.0
1.1625944419442848e-07
0.0
1.1625944419442848e-07
1.1625944419442848e-07
1.1625944419442848e-07
0.0
1
-5.4387125494463054e

In [95]:
n.weights

[array([[0.99982916, 0.99982916, 0.99982916],
        [0.99982916, 0.99982916, 0.99982916],
        [0.99982916, 0.99982916, 0.99982916],
        [0.99991458, 0.99991458, 0.99991458]]),
 array([[1.0053835 , 0.99456824],
        [1.0053835 , 0.99456824],
        [1.0053835 , 0.99456824],
        [1.00936329, 0.99047847]])]

In [30]:
# print("Loading train_x.csv, elapsed time: ", time.clock() - start_time)
# x = np.loadtxt("../data/train_x.csv", delimiter=",", dtype=int)
# print("Loaded train_x.csv. Loading train_y.csv, elapsed time: ", time.clock() - start_time)
# y = np.loadtxt("../data/train_y.csv", delimiter=",", dtype=int)
# print(y.shape)
# x = x.reshape(-1, 64*64)
# y = y.reshape(-1, 1)

x = np.load("../../centered_digits.npy")
y = np.loadtxt("../../train_y.csv", delimiter=",", dtype=int)
y = y.reshape(-1,)
x = x.reshape(-1, 64*64)

# print(type(y[0][0]), y[0].shape, y.shape)

# binary_pics = (x > 250) + 0
print("Training NN, elapsed time: ", time.clock() - start_time)
NN = Network([4096, 100, 100, 10])
NN.fit(x[:10000], y[:10000], 1)

print("NN trained. elapsed time: ", time.clock() -start_time)

# epochs = [1, 2, 4, 8]
# nhidden = [1, 2, 3]
# hiddensize = [1, 5, 50, 500, 4096]

# def tune(epochs, nhidden, hiddensize, train_x, train_y, test_x, test_y):
#     res = []
#     for nepochs in epochs:
#         for layers in nhidden:
#             for size in hiddensize:
#                 nodes = [4096]
#                 for layer in layers:
#                     nodes.append(size)
#                 nodes.append(10)
#                 NN = Network(nodes)
#                 NN.fit(train_x, train_y, nepochs)
#                 preds = NN.predict(test_x, test_y)
#                 s = score(preds, test_y, 4096)
#                 res.append((nepochs, layers, size, score))
#     return res



Training NN, elapsed time:  2218.861748
starting fit, time is:  2223.948909
epoch:  1
batch:  0
batch:  1
batch:  2
batch:  3
batch:  4
batch:  5
batch:  6
batch:  7
batch:  8
batch:  9
batch:  10
batch:  11
batch:  12
batch:  13
batch:  14
batch:  15
batch:  16
batch:  17
batch:  18
batch:  19
batch:  20
batch:  21
batch:  22
batch:  23
batch:  24
batch:  25
batch:  26
batch:  27
batch:  28
batch:  29
batch:  30
batch:  31
batch:  32
batch:  33
batch:  34
batch:  35
batch:  36
batch:  37
batch:  38
batch:  39
batch:  40
batch:  41
batch:  42
batch:  43
batch:  44
batch:  45
batch:  46
batch:  47
batch:  48
batch:  49
batch:  50
batch:  51
batch:  52
batch:  53
batch:  54
batch:  55
batch:  56
batch:  57
batch:  58
batch:  59
batch:  60
batch:  61
batch:  62
batch:  63
batch:  64
batch:  65
batch:  66
batch:  67
batch:  68
batch:  69
batch:  70
batch:  71
batch:  72
batch:  73
batch:  74
batch:  75
batch:  76
batch:  77
batch:  78
batch:  79
batch:  80
batch:  81
batch:  82
batch:  83


In [24]:
preds = NN.predict(x[40000:40100], y[40000:40100])
print( "predictions: ", preds, "score: ", score(preds, y[40000:40100], 10))

[[1.01206302 1.06893022 1.00633503 0.99962303 0.99509313 0.95457387
  0.99846011 1.00300417 1.00580525 0.95236217]
 [1.01206302 1.06893022 1.00633503 0.99962303 0.99509313 0.95457387
  0.99846011 1.00300417 1.00580525 0.95236217]
 [1.01206302 1.06893022 1.00633503 0.99962303 0.99509313 0.95457387
  0.99846011 1.00300417 1.00580525 0.95236217]
 [1.01206302 1.06893022 1.00633503 0.99962303 0.99509313 0.95457387
  0.99846011 1.00300417 1.00580525 0.95236217]
 [1.01206302 1.06893022 1.00633503 0.99962303 0.99509313 0.95457387
  0.99846011 1.00300417 1.00580525 0.95236217]
 [1.01206302 1.06893022 1.00633503 0.99962303 0.99509313 0.95457387
  0.99846011 1.00300417 1.00580525 0.95236217]
 [1.01206302 1.06893022 1.00633503 0.99962303 0.99509313 0.95457387
  0.99846011 1.00300417 1.00580525 0.95236217]
 [1.01206302 1.06893022 1.00633503 0.99962303 0.99509313 0.95457387
  0.99846011 1.00300417 1.00580525 0.95236217]
 [1.01206302 1.06893022 1.00633503 0.99962303 0.99509313 0.95457387
  0.99846011

In [33]:
NN.outputs[-1]

array([[4.61560101e+08, 4.61560101e+08, 4.61560101e+08, 4.61560101e+08,
        4.61560101e+08, 4.61560101e+08, 4.61560101e+08, 4.61560101e+08,
        4.61560101e+08, 4.61560101e+08],
       [3.72310101e+08, 3.72310101e+08, 3.72310101e+08, 3.72310101e+08,
        3.72310101e+08, 3.72310101e+08, 3.72310101e+08, 3.72310101e+08,
        3.72310101e+08, 3.72310101e+08],
       [7.59910101e+08, 7.59910101e+08, 7.59910101e+08, 7.59910101e+08,
        7.59910101e+08, 7.59910101e+08, 7.59910101e+08, 7.59910101e+08,
        7.59910101e+08, 7.59910101e+08],
       [7.75210101e+08, 7.75210101e+08, 7.75210101e+08, 7.75210101e+08,
        7.75210101e+08, 7.75210101e+08, 7.75210101e+08, 7.75210101e+08,
        7.75210101e+08, 7.75210101e+08],
       [2.44810101e+08, 2.44810101e+08, 2.44810101e+08, 2.44810101e+08,
        2.44810101e+08, 2.44810101e+08, 2.44810101e+08, 2.44810101e+08,
        2.44810101e+08, 2.44810101e+08],
       [5.61010101e+08, 5.61010101e+08, 5.61010101e+08, 5.61010101e+08,
   

In [28]:
NN.weights

[array([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
          1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
          1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
          1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
        ...,
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
          1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
          1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
        [-4.37354445e+03, -4.37354445e+03, -4.37354445e+03, ...,
         -4.37354445e+03, -4.37354445e+03, -4.37354445e+03]]),
 array([[-2.75002868e+06, -2.75002868e+06, -2.75002868e+06, ...,
         -2.75002868e+06, -2.75002868e+06, -2.75002868e+06],
        [-2.75002868e+06, -2.75002868e+06, -2.75002868e+06, ...,
         -2.75002868e+06, -2.75002868e

In [137]:
np.sum(NN.derivatives[-1])

0.0

In [30]:
np.sum(x[0])

288.65999999999997

In [15]:
def softmaxLoss( X, y):
        if (len(y.shape) == 0):
            m = 1
        else:
            m = y.shape[0]
        p = softmax(X)
#         log_likelihood = -np.log(p[range(m), y])
#         loss = np.sum(log_likelihood) / m

        dx = p.copy()
        dx[range(m), y] -= 1
        dx /= m
        return dx

def softmax( x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

In [17]:
softmaxLoss([[0.9,0.2], [0.3, 0.5]], np.array([0,1]))

array([[-0.16590611,  0.16590611],
       [ 0.225083  , -0.225083  ]])